# Bottleneck features

Notes: 

- If working in offline environment then need to save models in ~/.keras/models or it will automatically attempt to download
- I moved a few .jpeg images into a local folder for testing the code.

## Imports and paths

In [1]:
import numpy as np
import pandas as pd
import os
from os import listdir
from keras.preprocessing import image
from keras.applications import inception_v3, xception, vgg16, resnet50
import h5py as h5py
from tqdm import tqdm

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
INPUT_SIZE = 299
data_dir = '/notebooks/data_test/' #<-----Local foloder on my machine with a few jpgs in it.
num_records=len(listdir((data_dir)))
print("number of images: " + str(len(listdir((data_dir)))))

number of images: 5


In [3]:
#Creat list of image filepaths
image_path_list=[]
for path, subdirs, files in os.walk(data_dir):
    for name in files:
        image_path_list.append((os.path.join(path, name)))

In [4]:
def read_img(img_id, size):
    """Read and resize image.
    # Arguments
        img_id: string
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(img_id,target_size=size)
    img = image.img_to_array(img)
    return img

## Extract Features

Note that different models required different input sizes
- Xception (299x299)
- Inception (299x299)
- VGG (244x244)
- Res50 (244x244)

### Xception

In [5]:
# Read images to array
POOLING = 'avg'
x_train = np.zeros((num_records, INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in enumerate(tqdm(image_path_list)):
    img = read_img(img_id, (INPUT_SIZE, INPUT_SIZE))
    x = xception.preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

100%|██████████| 5/5 [00:00<00:00, 87.04it/s]

Train Images shape: (5, 299, 299, 3) size: 1,341,015


In [6]:
# Extract bottleneck features

xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
train_x_bf = xception_bottleneck.predict(x_train, batch_size=32, verbose=1)
print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))

5/5 [==============================] - 1s 240ms/step
Xception train bottleneck features shape: (5, 2048) size: 10,240


### Inception

In [7]:
POOLING = 'avg'
x_train = np.zeros((num_records, INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in enumerate(tqdm(image_path_list)):
    img = read_img(img_id, (INPUT_SIZE, INPUT_SIZE))
    x = inception_v3.preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

100%|██████████| 5/5 [00:00<00:00, 181.73it/s]

Train Images shape: (5, 299, 299, 3) size: 1,341,015


In [8]:
inception_bottleneck = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling=POOLING)
train_i_bf = inception_bottleneck.predict(x_train, batch_size=32, verbose=1)
print('InceptionV3 train bottleneck features shape: {} size: {:,}'.format(train_i_bf.shape, train_i_bf.size))

5/5 [==============================] - 2s 342ms/step
InceptionV3 train bottleneck features shape: (5, 2048) size: 10,240


### VGG

In [9]:
INPUT_SIZE = 224

In [10]:
POOLING = 'avg'
x_train = np.zeros((num_records, INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in enumerate(tqdm(image_path_list)):
    img = read_img(img_id, (INPUT_SIZE, INPUT_SIZE))
    x = vgg16.preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

100%|██████████| 5/5 [00:00<00:00, 170.03it/s]

Train Images shape: (5, 224, 224, 3) size: 752,640


In [11]:
vgg_bottleneck = vgg16.vgg16.VGG16(weights='imagenet', include_top=False, pooling=POOLING)
train_vgg_bf = vgg_bottleneck.predict(x_train, batch_size=32, verbose=1)
print('VGG train bottleneck features shape: {} size: {:,}'.format(train_vgg_bf.shape, train_vgg_bf.size))

5/5 [==============================] - 1s 247ms/step
VGG train bottleneck features shape: (5, 512) size: 2,560


### Res50

In [12]:
x_train = np.zeros((num_records, INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in enumerate(tqdm(image_path_list)):
    img = read_img(img_id, (INPUT_SIZE, INPUT_SIZE))
    x = resnet50.preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

100%|██████████| 5/5 [00:00<00:00, 181.76it/s]

Train Images shape: (5, 224, 224, 3) size: 752,640


In [13]:
res50_bottleneck = resnet50.resnet50.ResNet50(weights='imagenet', include_top=False, pooling=POOLING)
train_res50_bf = res50_bottleneck.predict(x_train, batch_size=32, verbose=1)
print('Res50 train bottleneck features shape: {} size: {:,}'.format(train_res50_bf.shape, train_res50_bf.size))

5/5 [==============================] - 1s 233ms/step
Res50 train bottleneck features shape: (5, 2048) size: 10,240


### Notes



- It is probably inefficient to read the images into separate arrays four times.
- I'm not sure it matters too much where preprocess_input is run from (i.e. VGG, Imagenet etc.)
- It may be possible to read the images into a 299 array and then simply resize down to 224 for the models that require  that size